In [ ]:
%pip install -U weaviate-client
%pip install python-dotenv  
%pip install pandas

In [1]:
import weaviate
import weaviate.classes as wvc
from weaviate.classes.init import Auth
from dotenv import load_dotenv
import json
import os
import requests
import pandas as pd

In [26]:
load_dotenv()

client = weaviate.connect_to_weaviate_cloud(
    cluster_url = os.getenv("CLUSTER_URL"),
    auth_credentials = weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
    headers = {
        "X-OpenAI-Api-Key": os.getenv("OPENAI_AI_KEY")
    }
)

In [ ]:
questions = client.collections.create(
    name="Question",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.config.Configure.Generative.openai()
)

In [7]:
df = pd.read_json('paragraphs_20240805_01.json', lines=True)

In [28]:
collection = client.collections.get("Question")
with collection.batch.dynamic() as batch:
    for i, row in df.iterrows():
        
        properties = {
            "answer": row[0],
            "book": row[1]
        }

        batch.add_object(properties=properties)

In [ ]:
#configure batch
client.batch.configure(batch_size=100)  
with client.batch as batch:
    
    

In [ ]:
question_objs = list()

for i in range(3001,4000):
    question_objs.append({'text':block_list[i]})

questions = client.collections.get("Question")
questions.data.insert_many(question_objs)
    

In [ ]:
try:
    pass
    questions = client.collections.get("Question")
    response = questions.query.near_text(
        query="What should I do this evening?",
        limit=2
    )

    print(f'{response.objects[0].properties}\n')
    print(response)

finally:
    client.close()
    